In [1]:
import cv2 as cv
import numpy as np
import epfl_data2 as epfl_data
import matplotlib.pyplot as plt
from skimage.feature import hog as HoG
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle


# import training_checkpoints as note

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def flip_angle(angle):
    if angle == 180:
        return 0
    elif angle == 0:
        return 180
    else:
        return 360-angle

# note.depends_loaded()

rotations = 3
bins = 6

data = epfl_data.Data(rotations,bins)

# print(data.samples[0])
# print(data.samples[1])
print(data.samples[2][0])
# print(data.samples[3])

training_images = np.zeros((1327,11016))
training_images_f = np.zeros((1327,11016))
test_images = np.zeros((972,11016))
training_tgts_1 = []
training_tgts_2 = []
training_tgts_3 = []
training_tgts_f1 = []
training_tgts_f2 = []
training_tgts_f3 = []
train_angles = []
train_angles_f = []
test_tgts = []

height = 250
width = 376

for d in range(len(data.samples[0])*2):
    image = cv.imread(data.samples[0][d])
    image = image[28:212, 30:348]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    f_image = np.fliplr(image)
    if d <= 1326:
        image = HoG(image, orientations=8, pixels_per_cell=(16,16))
        image = normalize(image)
        f_image = HoG(f_image, orientations=8, pixels_per_cell=(16,16))
        f_image = normalize(image)
        training_images[d] = image
        training_tgts_1.append(data.samples[2][d][0])
        training_tgts_2.append(data.samples[2][d][1])
        training_tgts_3.append(data.samples[2][d][2])
        training_images_f[d] = f_image
        training_tgts_f1.append(data.samples[2][d][0])
        training_tgts_f2.append(data.samples[2][d][1])
        training_tgts_f3.append(data.samples[2][d][2])
        
        train_angles.append(np.deg2rad(data.samples[1][d]))
        train_angles_f.append(np.deg2rad(flip_angle(data.samples[1][d])))
    else:
        if d-1326 == 972:
            break
        else:
            image = HoG(image, orientations=8, pixels_per_cell=(16, 16))
            image = normalize(image)
            test_images[d-1326] = image
            test_tgts.append(np.deg2rad(data.samples[1][d]))

print(training_images.shape)
train = np.append(training_images, training_images_f,axis=0)
print(train.shape)
# shuffled_training = shuffle(train)
for i in range(len(training_tgts_1)):
    training_tgts_1.append(training_tgts_f1[i])
    training_tgts_2.append(training_tgts_f2[i])
    training_tgts_3.append(training_tgts_f3[i])
# training_tgts.append(training_tgts_f)
for i in range(len(train_angles)):
    train_angles.append(train_angles_f[i])
print('done')

[[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.]]
(1327, 11016)
(2654, 11016)
done


In [3]:
print(data.samples[2][0])
print(data.samples[2][0][0])
p=np.zeros((2,2))
pp=np.ones((2,2))
print(np.append(p,pp,axis=0))

[[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.]]
[ 0.  0.  0.  0.  0.  1.]
[[ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  1.]]


In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras.losses import categorical_crossentropy as cross

Using TensorFlow backend.


In [5]:
rotations = 3

In [6]:
hog_input = Input(shape=(11016,), name='input')
layers = Dense(1000, input_dim=11016, activation='relu')(hog_input)
layers = Dropout(.5)(layers)
layers = Dense(100, activation='relu')(layers)
layers = Dropout(.5)(layers)
layers = Dense(10, activation='relu')(layers)
layers = Dropout(.5)(layers)
outputs = []
for r in range(rotations):
    outputs.append(Dense(bins, activation='softmax', name='output{}'.format(r))(layers))



In [7]:
print(outputs)

[<tf.Tensor 'output0/Softmax:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'output1/Softmax:0' shape=(?, 6) dtype=float32>, <tf.Tensor 'output2/Softmax:0' shape=(?, 6) dtype=float32>]


In [9]:
model = Model(inputs=hog_input, outputs=outputs)
model.compile(optimizer='adam', loss=cross, metrics=['accuracy'])
model.fit(train, 
          [np.asarray(training_tgts_1), np.asarray(training_tgts_2), np.asarray(training_tgts_3)], 
          batch_size=32, epochs=10, verbose=1)

Epoch 1/10
2654/2654 [==============================] - 45s - loss: 5.2374 - output0_loss: 1.7194 - output1_loss: 1.7697 - output2_loss: 1.7483 - output0_acc: 0.2200 - output1_acc: 0.1696 - output2_acc: 0.2163    
Epoch 2/10
2654/2654 [==============================] - 45s - loss: 4.7261 - output0_loss: 1.5300 - output1_loss: 1.6287 - output2_loss: 1.5674 - output0_acc: 0.3018 - output1_acc: 0.2743 - output2_acc: 0.3263    
Epoch 3/10
2654/2654 [==============================] - 45s - loss: 4.0565 - output0_loss: 1.2728 - output1_loss: 1.3941 - output2_loss: 1.3896 - output0_acc: 0.4480 - output1_acc: 0.3983 - output2_acc: 0.3971    
Epoch 4/10
2654/2654 [==============================] - 39s - loss: 3.2578 - output0_loss: 1.0259 - output1_loss: 1.0793 - output2_loss: 1.1525 - output0_acc: 0.5531 - output1_acc: 0.5437 - output2_acc: 0.5151    
Epoch 5/10
2654/2654 [==============================] - 39s - loss: 2.6532 - output0_loss: 0.8564 - output1_loss: 0.8782 - output2_loss: 0.9186 

In [10]:
second_round = model.predict(train)
print(second_round)

[array([[  4.16598376e-03,   1.81027676e-03,   2.09481679e-02,
          5.62100066e-03,   4.27533165e-02,   9.24701273e-01],
       [  7.93765247e-01,   1.90695226e-01,   1.28707010e-02,
          9.56767122e-04,   8.52657191e-04,   8.59512656e-04],
       [  9.09193277e-01,   8.84162635e-02,   2.17170408e-03,
          9.42640181e-05,   7.22466139e-05,   5.22943883e-05],
       ..., 
       [  9.21378851e-01,   7.14047626e-02,   5.91222476e-03,
          6.83030230e-04,   3.52063886e-04,   2.69079843e-04],
       [  9.36341941e-01,   5.70308976e-02,   5.30502899e-03,
          7.36289774e-04,   3.51459370e-04,   2.34448613e-04],
       [  9.51792479e-01,   4.29441594e-02,   4.14250651e-03,
          6.62988052e-04,   2.92333716e-04,   1.65487203e-04]], dtype=float32), array([[  2.48952117e-03,   1.91423651e-02,   8.58445093e-03,
          7.17468262e-02,   8.55214179e-01,   4.28226665e-02],
       [  3.04697067e-01,   3.69850383e-03,   3.63593455e-03,
          7.46080931e-03,   8.12

In [11]:
tester = 555
print(second_round[0][tester])
print(second_round[1][tester])
print("")
print(training_tgts_1[tester])
print(training_tgts_2[tester])
print(len(second_round[0]))
print(len(second_round[0][3]))

print(training_tgts_1)

[  6.78179879e-03   9.93211210e-01   6.94206437e-06   8.81819118e-10
   3.61103036e-09   3.01324099e-09]
[  9.79176939e-01   3.31735532e-06   2.03585074e-07   3.79881726e-06
   1.44006407e-08   2.08156500e-02]

[ 0.  1.  0.  0.  0.  0.]
[ 1.  0.  0.  0.  0.  0.]
2654
6
[array([ 0.,  0.,  0.,  0.,  0.,  1.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  1.,  0.,  0.,  0.,  0.]), array([ 0.,  1.,  0.,  0.,  0.,  0.]), array([ 0.,  1.,  0.,  0.,  0.,  0.]), array([ 0.,  1.,  0.,  0.,  0.,  0.]), array([ 0.,  1.,  0.,  0., 

In [53]:
second_array = np.asarray(second_round)
print(second_array.shape)
second_array = np.moveaxis(second_array, 1,0)
print(second_array.shape)
print(len(second_array))
print(len(second_array[1]))
print(len(second_array[:][0]))
# print(second_array[1][0])
# print(second_array[2][0])
# print(second_array[0][1])
# print(second_array[0][0])
flat_second = second_array.reshape(-1, second_array.shape[1])
# print(flat_second.shape)
# # print(flat_second)
# print('new')
# print(flat_second[0])
# print('new')
# print(flat_second[1])
# print('new')
# print(flat_second[2])
# print('new')
# print(flat_second[3])
# print('new')
# print(flat_second[4])

# probabilities=[]
# new_input = []
# for outs in range(len(second_array.shape[1])):
#     for i in range(rotations):
#         probabilities.append(list(second_round[i][outs]))
#     new_input.append([item for sublist in probabilities for item in sublist])
# print(new_input)

# test = np.asarray(([[1,1,1],[2,3,4,5],[6,6,6,6,6]]))
# print(test)

(3, 2654, 6)
(2654, 3, 6)
2654
3
3


In [51]:
# print(second_array[:][0][0])
print(flat_second.shape)
print(flat_second[0])

(15924, 3)
[ 0.00416598  0.00181028  0.02094817]


In [54]:
rot_input = Input(shape=(1,6,), dtype='float32', name='1_input')
rot_input2 = Input(shape=(1,6,), dtype='float32', name='2_input')
rot_input3 = Input(shape=(1,6,), dtype='float32', name='3_input')

layer = keras.layers.concatenate([rot_input, rot_input2, rot_input3])
layer = Dense(200, activation='relu')(layer)
layer = Dropout(.5)(layer)
layer = Dense(100, activation='relu')(layer)
layer = Dropout(.5)(layer)
layer = Dense(10, activation='relu')(layer)
layer = Dropout(.5)(layer)
output = Dense(1)(layer)

In [56]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

In [59]:
rot1 = tf.placeholder(tf.float32, [None, rotations,bins])
rot2 = tf.placeholder(tf.float32, [None, rotations,bins])
rot3 = tf.placeholder(tf.float32, [None, rotations,bins])
labelz = tf.placeholder(tf.float32, [None])
flatten1 = flatten(rot1)
flatten2 = flatten(rot2)
flatten3 = flatten(rot3)
W1 = tf.Variable(tf.zeros([2654, 100]))
B1 = tf.Variable(tf.zeros([100]))
Act1 = tf.nn.relu(tf.multiply(tf.concat(rot1,rot2,rot3),W1)+B1)
mse = tf.metics.mean_squared_error(labels=labelz, predictions = Act1)
optimizer = tf.trian.AdamOptimizer(.0001).minimize(mse)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(20):
        sess.run(optimizer, feed_dict={rot1:second_array[0][0], rot2:second_array[0][1], rot3:second_array[0][2]})


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [18]:
model_two = Model(inputs=[rot_input, rot_input2, rot_input3], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.fit([second_array[0][:],second_array[1][:],second_array[2][:]], 
          train_angles, 
          batch_size=32, epochs=10, verbose=1)
#shuffle the array?

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 arrays but instead got the following list of 3 arrays: [array([[  4.16598376e-03,   1.81027676e-03,   2.09481679e-02,
          5.62100066e-03,   4.27533165e-02,   9.24701273e-01],
       [  7.93765247e-01,   1.90695226e-01,   1.28707010e-02,
          9....

In [ ]:
score = model_two.evaluate(test_images[:-1], np.asarray(test_tgts), verbose=1)
print("\nSCORE:", score)
# note.all_complete()

model.summary()

predictions = model.predict(test_images[:-1])

print("PREDICTIONS:", predictions.T)
print("ACTUAL:", test_tgts)

In [ ]:
# model_two = Sequential()
# model_two.add(Dense(200), input_dim=rotations*bins, activation='relu')
# model_two.add(Dropout(.5))
# model_two.add(Dense(100), activation='relu')
# model_two.add(Dropout(.5))
# model_two.add(Dense(10), activation='relu')
# model_two.add(Dropout(.5))
# model_two.add(Dense(1), activation='sigmoid')

In [ ]:
model_two.fit(second, np.asarray(training_tgts), batch_size=32, epochs=20, verbose=1)

score = model.evaluate(test_images[:-1], np.asarray(test_tgts), verbose=1)
print("\nSCORE:", score)
# note.all_complete()

model.summary()

predictions = model.predict(test_images[:-1])

print("PREDICTIONS:", predictions.T)
print("ACTUAL:", test_tgts)